<center><h1> Support Vector Machines (SVM):</h1>
    <h2>Binary Classification Using a Hard-margin Linear Kernel</h2></center>
    
|Author| Stanley A. Baronett|
|--|-------------------------------|
|Updated | 12/8/2019|

### Abstract
Here

### Introduction
History, applications

### Theoretical Background
Here

### Methods
Here

### Results
Here

### Analysis
Here

### Interpretation and Conclusion
Here

Limitations of SVM:
